## Notes
My attempt at making a new interactive notebook.  
Main focus:  
    - Switching anndata objects  
    - Finding DEGs, probably precalculated table  
    - Plotting options  
        - Dotplot, featureplot  

Will start by trying to overhaul my interactive plotting code.  
First objective will be plotting a umap and being able to change the anndata object.  

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx, State
import dash_ag_grid as dag

import pandas as pd

import os
import sys
import time
import warnings
import gc
import io
import base64
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
ecto_interactive_nocycle = sc.read('h5ad_files/mouse/ecto_andrea/ecto_interactive_nocycle.h5ad')
adata_E8 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E8_split_ecto.h5ad')
adata_E9 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E9_split_ecto.h5ad')
adata_E10 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E10_split_ecto.h5ad')
adata_E11 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E11_split_ecto.h5ad')
adata_E12 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E12_split_ecto.h5ad')
adata_E13 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E13_split_ecto.h5ad')
adata_E14 = sc.read('h5ad_files/mouse/ecto_andrea/adata_E14_split_ecto.h5ad')


In [ ]:
anndata_objects = {'ecto_interactive_nocycle':ecto_interactive_nocycle,
                   'adata_E8':adata_E8,
                   'adata_E9':adata_E9,
                   'adata_E10':adata_E10,
                   'adata_E11':adata_E11,
                   'adata_E12':adata_E12,
                   'adata_E13':adata_E13,
                   'adata_E14':adata_E14,
                  }

In [ ]:
port = 11199
initial_gene = 'Car4'
initial_obs = 'sample'
initial_obsm = 'X_umap'

external_stylesheets = ['https://bootswatch.com/5/flatly/bootstrap.css']

app = Dash(__name__,
           external_stylesheets=external_stylesheets,
           suppress_callback_exceptions=True)


##########################################################
######################  Display of tab 1 #################
##########################################################
tab1 = html.Div([
    html.Div([
        html.Div([
            
            
            ##############################################
            
            ##############################################
            ######  select gene to plot from list  #######
            ##############################################
            html.Label("Select gene"),
            dcc.Dropdown(
                         id="dropdown_var",
                        value = initial_gene,
                        ),
            ##############################################
            
            ##############################################
            ####other gene values to show if available####
            ##############################################
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            ##############################################
            
            ##############################################
            #######set dot size of both umap plots########
            ##############################################
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=3,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
            ##############################################
        ],
            style={'width': '49%', 'display': 'inline-block'}), #html style of left dropdown section

        html.Div([
            ##############################################
            ######  select obsm to plot from list  #######
            ##############################################
            html.Label("Select obsm"),
            dcc.Dropdown(
                         id="dropdown_obsm",
                         value = initial_obsm
                        ),
            ##############################################
            
        ],
           style={'width': '20%', 'float': 'right', 'display': 'inline-block'}), #html style of right dropdown section
        html.Div([
            ##############################################
            ######  select obs to plot from list  #######
            ##############################################
            html.Label("Select obs"),
            dcc.Dropdown(
                         id="dropdown_obs",
                         value = initial_obs
                        ),
            ##############################################
            
        ],
           style={'width': '20%', 'float': 'right', 'display': 'inline-block'}), #html style of right dropdown section
        
    ],
        style={'padding': '10px 5px'}), #html style of dropdown section

    ##############################################
    ##########  plot left umap graph  ############
    ##############################################
    html.Div([
      dcc.Graph(id="graph1"),
        ],
        style={'width': '49%', 'display': 'inline-block'}),#html style of left graph section
    ##############################################
    
    ##############################################
    ##########  plot right umap graph  ###########
    ##############################################
    html.Div([
      dcc.Graph(id="graph2")
    ],
            style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),#html style of right graph section
    ##############################################

    ##############################################
    ############### DEG table stuff ##############
    ##############################################
    html.Div([dcc.Dropdown(id = 'cluster_select'),
             ],
             style={'width': '49%', 'display': 'inline-block','display': 'none'},
             id="html-cluster_select"),
    
    html.Div([dash_table.DataTable(filter_action="native",
                                   sort_action="native",
                                   sort_mode="multi",
                                   column_selectable="single",
                                   page_action="native",
                                   page_current= 0,
                                   page_size= 10,
                                   id = 'deg_table',
                                   ),
             ],
             style={'display': 'none'},
             id='html-deg_table'),
    ##############################################
    html.Div(dcc.Store(id='gene_plotted')),
])


##########################################################
######################  Display of DP tab ################
##########################################################

####################################
#####components of left DP col######
####################################

#components that span the entire left col
html_drop_dp_obs = html.Div([html.B("Select obs:"),
                             dcc.Dropdown(id="drop_dp_obs",
                                          style={'width':'80%'},
                                         ),
                            ],
                            style = {'display': 'none'},
                            id = 'html-drop_dp_obs')

html_drop_dp_method = html.Div([html.B("Select method:"),
                                dcc.Dropdown(id="drop_dp_method",
                                             options = ['t-test', 'wilcoxon', 't-test_overestim_var'], 
                                             value = 'wilcoxon',
                                             style={'width':'80%'},
                                            ),
                               ],
                               style = {'display': 'none'},
                               id = 'html-drop_dp_method')

html_dp_minfoldchange = html.Div([html.B("min fold change: "),
                                  dcc.Input(
                                      id = 'dp_minfoldchange',
                                      type='number',
                                      value=2.5,
                                      style={'width':'25%'},
                                  ),
                                 ],
                                 style = {'display': 'none'},
                                 id = 'html-dp_minfoldchange')

html_vmin_vmax = html.Div([html.B("vmin: "),
                           dcc.Input(id = 'dp_vmin',
                                     type='number',
                                     max = 0,
                                     value=-4,
                                     style={'width':'25%'},
                                    ),
                           html.Br(),
                           html.Br(),
                           
                           html.B("vmax: "),
                           dcc.Input(id = 'dp_vmax',
                                     type='number',
                                     min = 0,
                                     value=4,
                                     style={'width':'25%'},
                                    ),
                          ],
                          style = {'display': 'none'},
                          id = 'html-dp_vmin_vmax')

#components that span the left half of the left col
html_dp_var_type = html.Div([html.B("Genes: "),
                             dcc.RadioItems(id = 'dp_var_type',
                                            options = ['Automatic DEGs', 'Specify Genes'],
                                            value = 'Automatic DEGs',
                                            style = {'font-size': 12,},
                                           ),
                            ],
                            style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                            id = 'html-dp_var_type')

html_dp_n_genes = html.Div([html.Br(),
                            html.B("n_genes: "),
                            dcc.Input(id = 'dp_n_genes',
                                      type='number',
                                      min = 0,
                                      step = 1,
                                      value=4,
                                      style={'width':'50%'},
                                     ),
                           ],
                           style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                           id = 'html-dp_n_genes')

html_dp_select_obs = html.Div([html.B("Select categories"),
                               dcc.Dropdown(id="dp_select_obs",
                            #                options = list(anndata_objects[anndata].obs[drop_dp_obs].cat.categories),
                            #                value = list(anndata_objects[anndata].obs[drop_dp_obs].cat.categories),
                                            multi = True,
                                           ),
                              ],
                              style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                              id = 'html-dp_select_obs')

html_dp_select_var = html.Div([html.B("Select genes"),
                               dcc.Dropdown(
                                   id="dp_select_var",
                        #           options = anndata_objects[anndata].var.index.tolist(),
                                   multi = True,
                               ),
                              ],
                              style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                              id = 'html-dp_select_var')

#components that span the right half of the left col
html_dp_obs_type = html.Div([html.B("Categories: "),
                             dcc.RadioItems(id = 'dp_obs_type',
                                            options = ['Automatic order', 'Customize order'],
                                            value = 'Automatic order',
                                            style = {'font-size': 12,},
                                           ),
                            ],
                            style={'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'none'},
                            id = 'html-dp_obs_type')

html_dp_obs_order = html.Div([html.Div([html.Br(),
                                        dag.AgGrid(id="dp_obs_order",
                                                   dashGridOptions={"animateRows": True,
                                                                    "enableCellTextSelection": False,
                                                                    "skipHeaderOnAutoSize": True,
                                                                    #"ensureDomOrder": True,
                                                                    "rowDragManaged": True,
                                                                    #"rowDragMultiRow": True,
                                                                    #"rowSelection": "multiple",
                                                                    #"rowDragEntireRow": True,
                                                                    "pagination": False,
                                                                    "suppressRowClickSelection": True,
                                                                   }
                                                  )
                                       ],
                                       style={'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'none'},
                                       id = 'html-dp_obs_order'),
                              html.Div([dcc.Store(id='dp_obs_order_store',
                                                  data = None,
                                                 ),
                                       ])
                             ],)

####################################
####################################


####################################
#####components of right DP col#####
####################################

html_dp_figure = html.Div(children = [html.Img(id="dotplot",className="image", src = None, 
                                               style={'max-width': '100%','max-height':'60vh', 'display': 'inline-block'})],
                          id = 'html-dp_figure')

html_dp_button = html.Div([html.Button('Generate Plot', id='dp_button_press', n_clicks=0, className="btn btn-danger")],
                          style = {'display': 'none'},
                          id = 'html-dp_button')

####################################
####################################


html_dp_left_col = html.Div([html_drop_dp_obs,
                             html.Br(),
                             html_drop_dp_method,
                             html.Br(),
                             html_dp_minfoldchange,
                             html.Br(),
                             html_vmin_vmax,
                             html.Br(),
                             html_dp_var_type, #sub left
                             html_dp_obs_type, #sub right
                             html.Br(),html.Br(),
                             html_dp_n_genes,
                             html_dp_obs_order,
                             html.Br(),html.Br(),
                             html_dp_select_var,
                             html.Br(),html.Br(),
                             html_dp_select_obs,
                            ],
                            style={'width': '30%','height':'100%', 'float': 'left', 
                                   'display': 'inline-block','padding-left': '10px',
                                   'padding-bottom': '10px','padding-top': '10px','padding-right': '10px',
                                  },
                            className='bg-light',
                            id = 'html-dp_left_col')

html_dp_right_col = html.Div([html_dp_button,
                              html.Br(),
                              dcc.Loading([html_dp_figure],
                                          overlay_style={"visibility":"visible", "filter": "blur(2px)"},
                                          type="circle",),
                              #html_dp_figure,
                             ],
                             style={'width': '65%', 'float': 'right', 'display': 'inline-block','padding-left': '10px',
                                   'padding-bottom': '10px','padding-top': '10px','padding-right': '10px',},
                             id = 'html-dp_right_col')

tab2 = html.Div([html_dp_left_col,
                 html_dp_right_col, 
                ],
                style={'height':'100%'},
                id = 'html-tab2')

##############################################
##########  show items left tab    ###########
##############################################
@app.callback(
    Output('drop_dp_obs', 'options'),
    Output('html-drop_dp_obs','style'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output_drop_dp_obs(anndata):
    return list(anndata_objects[anndata].obs.dtypes[anndata_objects[anndata].obs.dtypes =='category'].index), {'display': 'block'}
########################
@app.callback(
    Output('html-drop_dp_method','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_drop_dp_method(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_button','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_button(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_minfoldchange','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_minfoldchange(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_vmin_vmax','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_vmin_vmax(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_var_type','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_var_type(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'block'}
########################
@app.callback(
    Output('html-dp_obs_type','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_obs_type(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'block'}
########################
@app.callback(
    Output('html-dp_n_genes','style'),
    Output('html-dp_select_obs','style'),
    Output('dp_select_obs','options'),
    Output('dp_select_obs','value'),
    Output('html-dp_select_var','style'),
    Output('dp_select_var','options'),
    Output('dp_minfoldchange','disabled'),
    Input('anndata', 'value'),
    Input('drop_dp_obs', 'value'),
    Input('dp_var_type', 'value'),
    prevent_initial_call=True
)
def update_output_dp_var_type_submenu(anndata,drop_dp_obs,dp_var_type):
    showstyle = {'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'block'}
    hidestyle = {'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'}
    if drop_dp_obs is not None:
        if dp_var_type == 'Automatic DEGs':
            new_options = list(anndata_objects[anndata].obs[drop_dp_obs].cat.categories)
            new_value = list(anndata_objects[anndata].obs[drop_dp_obs].cat.categories)
            return showstyle,showstyle,new_options,new_value,hidestyle,[],False
        elif dp_var_type == 'Specify Genes':
            new_options = anndata_objects[anndata].var.index.tolist()
            return hidestyle,hidestyle,[],[],showstyle,new_options,True
        else:
            return hidestyle,hidestyle,[],[],hidestyle,[],False
    else:
        return hidestyle,hidestyle,[],[],hidestyle,[],False
########################
@app.callback(
    Output('html-dp_obs_order','style'),
    Output('dp_obs_order','rowData'),
    Output('dp_obs_order','columnDefs'),
    Input('anndata', 'value'),
    Input('drop_dp_obs', 'value'),
    Input('dp_obs_type','value'),
    prevent_initial_call=True
)
def update_output_dp_obs_order(anndata,drop_dp_obs,dp_obs_type):
    hidestyle = {'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'none'}
    if drop_dp_obs is not None:
        if dp_obs_type == 'Customize order':
            showstyle = {'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'block'}
            df_list = pd.DataFrame(list(anndata_objects[anndata].obs[drop_dp_obs].cat.categories),columns=['order categories:'])
            rowData=df_list.to_dict("records")
            columnDefs=[{"field": i,'rowDrag': True} for i in df_list.columns]
            return showstyle,rowData,columnDefs
        else:
            return hidestyle,None,None
    else:
        return hidestyle,None,None

@app.callback(
    Output('dp_obs_order_store','data'),
    Input('dp_obs_type','value'),
    Input('dp_obs_order','virtualRowData'),
    prevent_initial_call=True
)
def update_obs_order_store(dp_obs_type,dp_obs_order):
    if dp_obs_type == 'Customize order':
        return dp_obs_order
    else:
        return None

##############################################
##############################################

##############################################
##########  generate right tab     ###########
##############################################

##############################################
##############################################

##########################################################
#################### app layout   ########################
##########################################################
app.layout = html.Div([
    ##############################################
    ####select anndata object from dictionary#####
    ##############################################
    html.B("Select object"),
    dcc.Dropdown(options = list(anndata_objects), 
                 id="anndata",
                ),
    ##############################################

    ##############################################
    ###############tabular layout#################
    ##############################################
    dcc.Tabs([
        dcc.Tab(tab1, label="Interactive UMAP"),
        dcc.Tab(tab2, label="Dotplot",),
        ]),
    ##############################################
],style={'height':'100%'})
##########################################################
##########################################################


##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 1 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################

##########################################################
##########set anndata object for other menus##############
##########################################################

@app.callback(
    Output('dropdown_var', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].var.index.tolist()

@app.callback(
    Output('dropdown_obs', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].obs.columns.tolist()

@app.callback(
    Output('dropdown_obsm', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return list(anndata_objects[anndata].obsm.keys())

##########################################################
##########################################################

##########################################################
########## plot expression of gene on umap  ##############
##########################################################
@app.callback(
    Output("graph1", "figure"),
    Output('dropdown_var','value'),
    Input('anndata', 'value'),
    Input("slider_marker_size", "value"),    
    Input("dropdown_var", "value"),
    Input('deg_table', 'active_cell'),
    Input('dropdown_obsm','value'),
    State('deg_table', 'derived_viewport_data'),
    prevent_initial_call=True
)

def update_graph_1(anndata,
                   slider_marker_size,
                   dropdown_var,
                   active_cell,
                   dropdown_obsm,
                   derived_viewport_data,
                  ):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    elif callback_id == 'deg_table':
        try:
            color_var = derived_viewport_data[active_cell['row']]['names']
        except:
            color_var = dropdown_var
    else:
        color_var = dropdown_var

    # if anndata not in anndata_objects.keys():
    #     fig = None

    if anndata in anndata_objects.keys():
        tmp_df = pd.DataFrame(anndata_objects[anndata].obsm[dropdown_obsm][:,:2], columns = ['X','Y'])
        tmp_df.index = anndata_objects[anndata].obs.index
        tmp_df = tmp_df.join(anndata_objects[anndata].obs)
        tmp_df['barcode'] = list(tmp_df.index)
        plot_df = tmp_df
        fig = px.scatter(plot_df, x='X', y='Y',
                         color = anndata_objects[anndata][:,color_var].X.toarray().flatten(),
                         hover_data={'X':False,
                                     'Y':False,
                                     'barcode':True,
                                     'sample':True,
                                     'phase':True,},
                         color_continuous_scale = plotly_reds,
                         labels = {'color':color_var},
                         width=700, height=700,
                        )
        fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                                 })
        fig.update_layout(uirevision='constant')

    return fig, color_var

##########################################################
##########################################################

##########################################################
########## plot annotation of obs  on umap  ##############
##########################################################
@app.callback(
    Output("graph2", "figure"),
    Input("dropdown_obs", "value"),
    Input("slider_marker_size", "value"),
    Input('anndata', 'value'),
    Input('dropdown_obsm','value'),
    prevent_initial_call=True
)
def update_graph_2(dropdown_obs, slider_marker_size,anndata,dropdown_obsm):
    tmp_df = pd.DataFrame(anndata_objects[anndata].obsm[dropdown_obsm][:,:2], columns = ['X','Y'])
    tmp_df.index = anndata_objects[anndata].obs.index
    tmp_df = tmp_df.join(anndata_objects[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    sorted_list = []
    for i in plot_df:
        if pd.api.types.is_categorical_dtype(plot_df[i]):
            plot_df[i] = plot_df[i].cat.remove_unused_categories()
    if pd.api.types.is_categorical_dtype(plot_df[dropdown_obs]):
        try:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories), key = int)
        except:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories))
                             
     
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = plot_df[dropdown_obs],
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        #color='leiden_0.5',
                        color_continuous_scale = plotly_reds,
                        category_orders={dropdown_obs: sorted_list},
                        width=800, height=700,
                       )
    
    fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(legend= {'itemsizing': 'constant'})

    fig.update_layout(uirevision='constant')
    return fig
##########################################################
##########################################################

##########################################################
#################### Update DEG table  ###################
##########################################################
@app.callback(Output('html-cluster_select','style'),
              Output('cluster_select','options'),
              Output('cluster_select','value'),
              Input('dropdown_obs','value'),
              Input('anndata', 'value'), 
              prevent_initial_call=True
             )
def cluster_dropdown(dropdown_obs,anndata):
    if 'wilcoxon_'+dropdown_obs in anndata_objects[anndata].uns.keys():
        style = {'width': '49%', 'display': 'inline-block','display': 'block'}
        options = list(anndata_objects[anndata].obs[dropdown_obs].cat.categories)
        value = list(anndata_objects[anndata].obs[dropdown_obs].cat.categories)[0]
    else:
        style = {'width': '49%', 'display': 'inline-block','display': 'none'}
        options = []
        value = None
    return style,options,value

@app.callback(Output('html-deg_table','style'),
              Output('deg_table','data'),
              Output('deg_table','columns'),
              Input('dropdown_obs','value'),
              Input('anndata','value'),
              Input('cluster_select','value'),
              prevent_initial_call=True
             )
def update_datatable(dropdown_obs,anndata,cluster_select):            
    if 'wilcoxon_'+dropdown_obs in anndata_objects[anndata].uns.keys():
        style = {'display': 'block'}
        deg_df = sc.get.rank_genes_groups_df(anndata_objects[anndata], group=None, key='wilcoxon_'+dropdown_obs)
        deg_df = deg_df[deg_df['group']==cluster_select].iloc[:,1:6]
        data = deg_df.to_dict('records')
        columns =  [{"name": i, "id": i,} for i in (deg_df.columns)]
    else:
        style = {'display': 'none'}
        data = None
        columns = None
    return style,data,columns
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 2 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################



##########################################################
############     generate dotplot         ################
##########################################################

@app.callback(
    Output('dotplot', 'src'),
   # Output('dp_button_press', 'n_clicks'),
    Input('dp_button_press', 'n_clicks'),
    State('anndata', 'value'),
    State('drop_dp_obs','value'),
    State('drop_dp_method','value'),
    State('dp_minfoldchange','value'),
    State('dp_vmin','value'),
    State('dp_vmax','value'),
    State('dp_var_type','value'),
    State('dp_n_genes','value'),
    State('dp_select_obs','value'),
    State('dp_select_var','value'), 
    State('dp_obs_type','value'),
    State('dp_obs_order_store','data'),
    State('dotplot', 'src'),
    prevent_initial_call=True
)
def generate_html_dp_figure(dp_button_press,
                            anndata,
                            drop_dp_obs,
                            drop_dp_method,
                            dp_minfoldchange,
                            dp_vmin,
                            dp_vmax,
                            dp_var_type,
                            dp_n_genes,
                            dp_select_obs,
                            dp_select_var,
                            dp_obs_type,
                            dp_obs_order_store,
                            dotplot_src,
                           ):
    if dp_button_press > 0:
        if drop_dp_method+'_'+drop_dp_obs not in anndata_objects[anndata].uns:
            sc.tl.rank_genes_groups(anndata_objects[anndata], 
                                    groupby = drop_dp_obs,
                                    method = drop_dp_method,
                                    key_added = drop_dp_method+'_'+drop_dp_obs,
                                   )
            
        if dp_obs_type == 'Automatic order':
            if 'dendrogram_'+drop_dp_obs not in anndata_objects[anndata].uns:
                sc.tl.dendrogram(anndata_objects[anndata],
                                 groupby = drop_dp_obs,
                                )
            categories_order = None
            dendrogram = True

        if dp_obs_type == 'Customize order':
            categories_order = [list(d.values())[0] for d in dp_obs_order_store]
            dendrogram = False

        if dp_var_type == 'Automatic DEGs':
            n_genes = dp_n_genes
            select_obs = dp_select_obs
            min_logfoldchange = dp_minfoldchange
            var_names = None

        if dp_var_type == 'Specify Genes':
            n_genes = None
            select_obs = None
            min_logfoldchange = None
            var_names = dp_select_var
            
        fig_buffer = io.BytesIO()
        dp = sc.pl.rank_genes_groups_dotplot(anndata_objects[anndata], 
                                             groups = select_obs,
                                             groupby = drop_dp_obs,
                                             n_genes=n_genes,
                                             var_names = var_names,
                                             categories_order = categories_order,
                                             dendrogram = dendrogram,
                                             values_to_plot="logfoldchanges",
                                             cmap='bwr',
                                             vmin=dp_vmin,
                                             vmax=dp_vmax,
                                             min_logfoldchange=min_logfoldchange,
                                             key = drop_dp_method+'_'+drop_dp_obs,
                                             colorbar_title='log fold change',
                                             return_fig = True)
        dp.savefig(fig_buffer, format='svg')
        plt.close()
        encoded = base64.b64encode(fig_buffer.getvalue()).decode("utf-8")
        fig_src = "data:image/svg+xml;base64, " + encoded
        
        #n_clicks = 0
        #n_clicks = dp_button_press
    else:
        fig_src = dotplot_src
       # n_clicks = dp_button_press
    return fig_src
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

if __name__ == '__main__':
    app.run(jupyter_mode="tab",host='0.0.0.0', port = port ,debug=True)
print('Use this link: '+'http://cranio213:'+str(port))

In [ ]:
# ############
# #Annotation by Andrea

# ## Leiden3
# # C9 Neuronal progenitors 1 (NP1): Neurog1, Neurod1
# # C12 Neuronal progenitors 2 (NP2): Neurod1, Lhx2
# # C37 Pioneer neurons?
# # C31 C31 Socs2,Eya5, Maob/Pcp4

# ## Leiden 2.4
# # C23 Neuronal progenitors 3 (NP3): Neurod1, Lhx2, Tesc, Col9a2
# # C28 Immature olfactory neurons1 (iOSN1): Lhx2, Tex15, S100z, Col9a2


# ## Leiden3
# # C18 Immature olfactory neurons2 (iOSN2): Gap43, Fstl5, Gng13
# # C0 + C33 + C36 Neuronal progenitors: Ascl1, Runx1
# # C32 VNO Ascl1 progenitors: Ascl1, Insm1, Fezf2
# # C14-> Ribosomal genes, out?
# # C29-> ??
# # C35-> mesenchyme, out?

# ## Leiden 1

# # C0 Aldh1a3-1: Aldh1a3, Necab2
# # C10 Aldh1a3-2: Aldh1a3, Agr3, Gria3, Trpm3
# # C12 Rim1: Fgf8, Dlk1, Bmp4
# # C14 Wnt5a, Trp63
# # C2 Rim2: Fgf8, Pax3, Trp63, Wnt6, Fxyd3, Grhl3, Krt15      
# # C5 Otx2, Agr3
# # C7 Maob/Pcp4 Progenitors: Maob, Pcp4
# # C16-> out?    

# ## Leiden 2.4
# # C3 Respiratory Epi. 1: Foxa1, Fxyd3, Olfm5
# # C22: Respiratory Epi. 2: Foxa1, Sult1e1


# ## Leiden 2
# # C24 Anterior forebrain: Slc2a3, Irs4, Car4, Car2
# # C22 Olfactory placode + early anteromedial: Six6, Dlk1, Vax1, Flrt2, Dlx5
# # C6 Olfactory placode + early dorsolateral: Pax3 (lateral), Dlk1, Sox2ot, Irs4, Eya4 (dorsal)
# # C3 Aldh1a3-3: Aldh1a3, Lgi2, Olfm5
# # C13 Aldh1a3-4:  Aldh1a3, Gata3, Krt14, Krt5
# # C1 VNO: Fezf2, Flrt2, car2, Mecom, Dlx5

# anndata_objects['ecto_interactive_nocycle'].obs['andrea_annotation'] = '- no_annotation -'

# #leiden 1
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '0'].obs.index,'andrea_annotation'] = 'Aldh1a3-1'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '10'].obs.index,'andrea_annotation'] = 'Aldh1a3-2'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '12'].obs.index,'andrea_annotation'] = 'Rim1'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '14'].obs.index,'andrea_annotation'] = 'C14 Wnt5a, Trp63'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '2'].obs.index,'andrea_annotation'] = 'Rim2'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '5'].obs.index,'andrea_annotation'] = 'C5 Otx2, Agr3'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '7'].obs.index,'andrea_annotation'] = 'Maob/Pcp4 Progenitors'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden1'] == '16'].obs.index,'andrea_annotation'] = 'out?'

# #leiden2
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2'] == '24'].obs.index,'andrea_annotation'] = 'Anterior forebrain'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2'] == '22'].obs.index,'andrea_annotation'] = 'Olfactory placode + early anteromedial'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2'] == '6'].obs.index,'andrea_annotation'] = 'Olfactory placode + early dorsolateral'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2'] == '3'].obs.index,'andrea_annotation'] = 'Aldh1a3-3'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2'] == '13'].obs.index,'andrea_annotation'] = 'Aldh1a3-4'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2'] == '1'].obs.index,'andrea_annotation'] = 'VNO'

# #leiden2.4
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2.4'] == '23'].obs.index,'andrea_annotation'] = 'Neuronal progenitors 3 (NP3)'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2.4'] == '28'].obs.index,'andrea_annotation'] = 'Immature olfactory neurons1 (iOSN1)'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2.4'] == '3'].obs.index,'andrea_annotation'] = 'Respiratory Epi. 1'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden2.4'] == '22'].obs.index,'andrea_annotation'] = 'Respiratory Epi. 2'


# #leiden3
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '9'].obs.index,'andrea_annotation'] = 'Neuronal progenitors 1 (NP1)'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '12'].obs.index,'andrea_annotation'] = 'Neuronal progenitors 2 (NP2)'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '18'].obs.index,'andrea_annotation'] = 'Immature olfactory neurons2 (iOSN2)'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '0'].obs.index,'andrea_annotation'] = 'Neuronal progenitors'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '33'].obs.index,'andrea_annotation'] = 'Neuronal progenitors'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '36'].obs.index,'andrea_annotation'] = 'Neuronal progenitors'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '32'].obs.index,'andrea_annotation'] = 'VNO Ascl1 progenitors'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '14'].obs.index,'andrea_annotation'] = 'Ribosomal genes, out?'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '29'].obs.index,'andrea_annotation'] = '??'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '35'].obs.index,'andrea_annotation'] = 'mesenchyme, out?'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '31'].obs.index,'andrea_annotation'] = 'C31 Socs2'
# anndata_objects['ecto_interactive_nocycle'].obs.loc[anndata_objects['ecto_interactive_nocycle'][anndata_objects['ecto_interactive_nocycle'].obs['leiden3'] == '37'].obs.index,'andrea_annotation'] = 'Pioneer neurons?'

# anndata_objects['ecto_interactive_nocycle'].obs['andrea_annotation'] = anndata_objects['ecto_interactive_nocycle'].obs['andrea_annotation'].astype('category')

# sc.tl.rank_genes_groups(anndata_objects['ecto_interactive_nocycle'], 'andrea_annotation', method='wilcoxon', 
#                              key_added = "wilcoxon_"+'andrea_annotation')

# sc.pl.umap(anndata_objects['ecto_interactive_nocycle'], color = 'andrea_annotation')

# anndata_objects['ecto_interactive_nocycle'].write('h5ad_files/mouse/ecto_andrea/ecto_interactive_nocycle.h5ad')